Analyzing Semi-Structured JSON Data with ChatGPT and Python
This notebook demonstrates how to work with semi-structured JSON data using both traditional Python tools and the ChatGPT API. It addresses key challenges such as:

1. Nested hierarchies
2. Inconsistent schemas
3. Mixed data types
4. Complex relationships
Objectives:
1. Extract specific values using natural language
2. Infer/document implicit schemas
3. Reformat content for downstream analysis
4. Generate Python code to process semi-structured data

In [1]:
# Load and inspect the JSON file
# Update your path below
import json
from pathlib import Path

json_path = Path('/content/sample_data/example_nested.json')
with open(json_path) as f:
    data = json.load(f)

# View the data
data

[{'order_id': 'A1001',
  'customer': {'id': 'C001', 'name': 'Alice Johnson'},
  'items': [{'product': 'Widget A', 'quantity': 2, 'price': 19.99},
   {'product': 'Widget B', 'quantity': 1, 'price': 29.99}],
  'order_total': 69.97,
  'status': 'shipped'},
 {'order_id': 'A1002',
  'customer': {'id': 'C002', 'name': 'Bob Smith'},
  'items': [{'product': 'Widget A', 'quantity': 1, 'price': 19.99}],
  'order_total': 19.99,
  'status': 'processing'}]

Traditional Parsing with pandas and json_normalize

In [2]:
import pandas as pd

# Flatten the nested JSON
df = pd.json_normalize(data,
                       record_path='items',
                       meta=['order_id', 'order_total', 'status', ['customer', 'id'], ['customer', 'name']],
                       errors='ignore')

df.head()

,product,quantity,price,order_id,order_total,status,customer.id,customer.name
0,Widget A,2,19.99,A1001,69.97,shipped,C001,Alice Johnson
1,Widget B,1,29.99,A1001,69.97,shipped,C001,Alice Johnson
2,Widget A,1,19.99,A1002,19.99,processing,C002,Bob Smith


In [3]:
import os
os.environ["OPENAI_API_KEY"]= "insert your key here"

In [4]:
import openai
import pandas as pd


In [5]:
import os
import openai
import pandas as pd
import json

# Function to prompt ChatGPT with dataset insights
def ask_chatgpt2(question, json_string):
    try:
        # Convert JSON string to Python object
        json_obj = json.loads(json_string)

        # Flatten the JSON if needed and convert to DataFrame
        if isinstance(json_obj, dict):
            df = pd.json_normalize(json_obj)
        elif isinstance(json_obj, list):
            df = pd.json_normalize(json_obj)
        else:
            raise ValueError("Unsupported JSON structure")

        # Limit data to first 100 rows for brevity
        dataset_str = df.head(100).to_string(index=False)

        # Construct prompt
        prompt = f"Here is the dataset:\n{dataset_str}\n\nQuestion: {question}"

        # Call OpenAI API
        response = openai.chat.completions.create(
            model="gpt-4-turbo",
            messages=[
                {"role": "system", "content": "You are a data analyst with expertise in Python, data visualization, and statistical analysis."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=150,
            temperature=0.7
        )

        return response.choices[0].message.content.strip()

    except Exception as e:
        return f"Error processing JSON: {e}"


In [6]:
# Load JSON file and convert to string
with open("/content/sample_data/example_nested.json", "r") as f:
    json_data_string = f.read()

question = "Analyze the following nested JSON structure and describe the schema, field types, and relationships"
response = ask_chatgpt2(question, json_data_string)
print(response)


The dataset you provided appears to be a table, with each row representing an order made by a customer. The fields and their types, based on the information provided, can be inferred as follows:

1. **order_id** (String): A unique identifier for each order (e.g., 'A1001', 'A1002').

2. **items** (List of Dictionaries/JSON Objects): This field contains details about the products ordered. Each item in the list is a dictionary with the following keys:
   - **product** (String): The name of the product (e.g., 'Widget A', 'Widget B').
   - **quantity** (Integer): The number of units of the product ordered.
   - **


In [9]:
question = "Write and provide the executable Python code to flatten the following JSON structure for analysis. Ensure that nested customer info and items are merged appropriately"
response = ask_chatgpt2(question, json_data_string)
print(response)

To flatten the JSON-like data you provided, we can use Python with libraries like `pandas` and `json_normalize` from `pandas.io.json`. The dataset as you described seems to be in a structured form that includes nested elements, particularly in the `items` field.

Below is the Python code that flattens the dataset and merges customer information and items:

```python
import pandas as pd
from pandas import json_normalize

# Define the dataset
data = [
    {
        "order_id": "A1001",
        "items": [
            {"product": "Widget A", "quantity": 2, "price": 19.99},
            {"product": "Widget B", "quantity": 1, "
